I have added some Visum related routines. Not to get all functional.
More to give example of different technics and the impact for run time.
I repeat some cells with different methods to evaluate the running time


In [12]:
import os.path

import win32com.client
import traceback
import win32com.client
import traceback
import sys
import importlib
import pandas as pd
import numpy as np
import platform

# Since we run in different environment, we need to handle absolute paths somehow.
# My suggestion is to use the computer name as an indication of the environment.
# If you implement in similar way in your code, I will be able to run the code without altering it

computer_name = platform.node()  # Use computer name to configure data for different run environments
# Specify the path to the helpers.py file
if computer_name == 'Josef-pc':
    helpers_path = r"C:\Program Files\PTV Vision\PTV Visum 2024\Exe\Python\Lib\site-packages\VisumPy\helpers.py"
else:
    helpers_path = 'D:\\PTV Visum 2024\\Exe\\Python\\Lib\\site-packages\\VisumPy\\helpers.py'

# Load the module specified by the path
spec = importlib.util.spec_from_file_location("helpers", helpers_path)
helpers = importlib.util.module_from_spec(spec)
sys.modules["helpers"] = helpers
spec.loader.exec_module(helpers)

# work path
if computer_name == 'Josef-pc':
    work_path = r"D:\Users\Joffa\PycharmProjects\AH203X_Degree_Project_in_Transport_Science\Mandl VISUM\Josef_test"
else:
    work_path = r"C:\Users\jespe\Documents\Python\GitHub\AH203X_Degree_Project_in_Transport_Science\Mandl VISUM\Josef_test"
# Path to VISUM file
visum_file_path = os.path.join(work_path, 'Mandl_Visum_set_runtimes.ver')
# Uses Mandl_Visum_set_runtimes.ver as version file.
# The only difference I made in the version file is that I added a procedure step that sets the lines run times from the link run time.
# Then you do not need to set in an accurate travel time for the time profile items


# Create a Visum instance
# Visum = win32com.client.Dispatch("Visum.Visum")
Visum = win32com.client.gencache.EnsureDispatch('Visum.Visum')  # To generate constants

# Load the VISUM file
Visum.LoadVersion(visum_file_path)

# Access the network object
net = Visum.Net

com_error: (-2147352567, 'Undantag inträffade.', (0, 'Visum.Visum.2401', 'CComIO::LoadVersion failed. File: C:\\Users\\jespe\\Documents\\Python\\GitHub\\AH203X_Degree_Project_in_Transport_Science\\Mandl VISUM\\Josef_test\\Mandl_Visum.ver', None, 0, -2147352567), None)

In [4]:
# Retrieve an existing line by its name or number
line_name_or_number = '1'
line = net.Lines.ItemByKey(line_name_or_number)
# Display information about the line
print(f"Line Name: {line.AttValue('Name')}")
print(f"Transport System: {line.AttValue('TSysCode')}")
print(f"Number of stops: {line.AttValue('StopsServed')}")

#Network information
#Line
for Line in Visum.Net.Lines:
    # Access attributes here
    Line_name = Line.AttValue('Name')
    print(f"Lines Name: {Line_name}")

#line route
for LineRoute in Visum.Net.LineRoutes:
    # Access attributes here
    LineRoute_name = LineRoute.AttValue('Name')
    print(f"LinesRoute Name: {LineRoute_name}")
    #TO CHANGE NAME OF A ROUTE

    #if LineRoute_name == "1":
    #    print("change name")
    #    LineRoute.SetAttValue('Name', "101")

#stops
for stop in Visum.Net.StopPoints:
    # Access attributes here
    stop_name = stop.AttValue('No')
    print(f"Stop Name: {stop_name}")

print("")

Line Name: 1
Transport System: B
Number of stops: 15.0
Lines Name: 1
LinesRoute Name: 101
LinesRoute Name: 102
Stop Name: 1.0
Stop Name: 2.0
Stop Name: 3.0
Stop Name: 4.0
Stop Name: 5.0
Stop Name: 6.0
Stop Name: 7.0
Stop Name: 8.0
Stop Name: 9.0
Stop Name: 10.0
Stop Name: 11.0
Stop Name: 12.0
Stop Name: 13.0
Stop Name: 14.0
Stop Name: 15.0



In [83]:
stop = net.StopPoints.ItemByKey(15)
# Display information about the line
print(f"stop Name: {stop.AttValue('No')}")

stop Name: 15.0


In [10]:
#REMOVE ALL BUSLINES
def remove_line_routes():
    for LineRoute in Visum.Net.LineRoutes:
        # Access attributes here
        Visum.Net.RemoveLineRoute(LineRoute)


remove_line_routes()
import random


def generate_bus_routes(all_stops, num_routes, max_stops_per_route):
    random.shuffle(all_stops)  # Shuffle the list of stops for randomness
    routes = []

    for _ in range(num_routes):
        route_length = random.randint(2, max_stops_per_route)  # Determine the length of the route
        route = random.sample(all_stops, route_length)  # Generate the route
        routes.append(route)
        routes.append(route[::-1])  # Append the reverse route

    return routes


#Collect all stops in network
def get_all_stop_no():
    all_stops = []  # Initialize an empty list to store line route names
    # Iterate over all LineRoutes in the Visum network
    for stop in Visum.Net.StopPoints:
        # Access the 'No' attribute of each Stop
        stop_no = stop.AttValue('No')
        # Add the name to the list
        all_stops.append(stop_no)

    return all_stops  # Return the list of line route names


#Collect all stops in network
def get_all_stop_no_faster():
    """Returns all stop point numbers

    prerequisite:
        a global variable Visum must be defined
"""
    return helpers.GetMulti(Visum.Net.StopPoints, 'No')


def create_route(route_vector, bus_nr):
    # use constants
    C = win32com.client.constants

    #retreive some required objects from the network
    BusLine = Visum.Net.Lines.ItemByKey("1")
    dirTo = Visum.Net.Directions.GetAll[0]
    stopsR1 = Visum.CreateNetElements()
    for stop in route_vector:
        stopPoint = Visum.Net.StopPoints.ItemByKey(stop)
        stopsR1.Add(stopPoint)

    paraR1 = Visum.IO.CreateNetReadRouteSearchTSys()  #create the parameter object
    paraR1.SetAttValue("HowToHandleIncompleteRoute", 3)  # search shortest path if line route has gaps
    paraR1.SetAttValue("ShortestPathCriterion", 3)  # link travel time of current transport system
    paraR1.SetAttValue("IncludeBlockedLinks", False)  # don't route over closed links
    paraR1.SetAttValue("IncludeBlockedTurns", False)  # don't route over closed turns
    paraR1.SetAttValue("MaxDeviationFactor",
                       1000)  # maximum deviation factor of shortest path search from direct distance
    paraR1.SetAttValue("WhatToDoIfShortestPathNotFound", 0)  # if no shortest path is found, don't read

    LineRoute = Visum.Net.AddLineRoute(bus_nr, BusLine, dirTo, stopsR1, paraR1)  #create the line route

    start_time = 0  # Start at 0 minutes (midnight)
    end_time = 24 * 60 * 60  # End at 86 400seconds or 1440 minutes (24 hours)
    frequency_seconds = 10 * 60

    # Create a time profile for the line route
    TimeProfileName = "TP" + str(bus_nr)
    tp1 = Visum.Net.AddTimeProfile(TimeProfileName, LineRoute)
    i = 1
    for time_in_seconds in range(start_time, end_time, frequency_seconds):
        trip1 = Visum.Net.AddVehicleJourney(bus_nr + str(i), tp1)
        trip1.SetAttValue("Dep", time_in_seconds)
        i += 1


def generate_Visum_network(num_routes, max_stops_per_route):
    all_stops = get_all_stop_no()
    bus_routes = generate_bus_routes(all_stops, num_routes, max_stops_per_route)
    for i in range(num_routes):
        if i >= 10:
            bus_nr = "1" + str(i + 10)
        else:
            bus_nr = "11" + str(i)
        create_route(bus_routes[i], bus_nr)


def load_net_file(net_file):
    # use constants
    C = win32com.client.constants

    # create AddNetRead-Object and specify desired conflict avoiding method
    anrController = Visum.IO.CreateAddNetReadController()
    anrController.SetWhatToDo("Line", C.AddNetRead_Ignore)
    anrController.SetWhatToDo("LineRoute", C.AddNetRead_Ignore)
    anrController.SetWhatToDo("LineRouteItem", C.AddNetRead_Ignore)
    anrController.SetWhatToDo("TimeProfile", C.AddNetRead_Ignore)
    anrController.SetWhatToDo("TimeProfileItem", C.AddNetRead_Ignore)
    anrController.SetWhatToDo("VehJourney", C.AddNetRead_Ignore)
    anrController.SetUseNumericOffset("VehJourney", True)
    anrController.SetWhatToDo("VehJourneyItem", C.AddNetRead_DoNothing)
    anrController.SetWhatToDo("VehJourneySection", C.AddNetRead_Ignore)
    anrController.SetWhatToDo("ChainedUpVehJourneySection", C.AddNetRead_DoNothing)
    anrController.SetWhatToDo("UserAttDef", C.AddNetRead_Ignore)
    anrController.SetWhatToDo("Operator", C.AddNetRead_OverWrite)

    # anrController.SetConflictAvoidingForAll(10000, "ORG_")

    # create NetRouteSearchTSys-Object and choose route search options
    # create one object per TSys if desired
    routesearchparameters = Visum.IO.CreateNetReadRouteSearchTSys()
    routesearchparameters.SetAttValue("HowToHandleIncompleteRoute",
                                      C.RouteSearchHandleIncompleteRouteTSearchShortestPath)  # search shortest path if line route has gaps
    routesearchparameters.SetAttValue("ShortestPathCriterion",
                                      C.ShortestPathCriterion_LinkLength)  # Value 4: Link length
    routesearchparameters.SetAttValue("IncludeBlockedLinks", False)  # don't route over closed links
    routesearchparameters.SetAttValue("IncludeBlockedTurns", False)  # don't route over closed turns
    routesearchparameters.SetAttValue("MaxDeviationFactor",
                                      1000)  # maximum deviation factor of shortest path search from direct distance
    routesearchparameters.SetAttValue("WhatToDoIfShortestPathNotFound",
                                      C.IfNotFound_DontRead)  # if no shortest path is found, don't read
    routesearchparameters.SetAttValue("WhatToDoIfStopPointIsBlocked",
                                      C.ifStopPointIsBlocked_DontReadTimeProfile)  # if stop point is blocked, don't read
    routesearchparameters.SetAttValue("WhatToDoIfStopPointNotFound",
                                      C.ifStopPointNotFound_DontReadLineRoute)  # if stop point not found, don't read

    # create NetRouteSearch-Object and assign NetRouteSearchTSys-objects
    nrrsController = Visum.IO.CreateNetReadRouteSearch()
    nrrsController.SetForAllTSys(routesearchparameters)

    #Load net file
    Visum.IO.LoadNet(net_file, True, nrrsController, anrController)


def save_as_net_file(bus_routes, bus_nr, net_file):
    """
    Saves the timetable as a net file

    Not implemented yet.
    Just a stub

    :param bus_routes: list of list of traversed stop points
    :param bus_nr: list of bus name
    :param net_file: filename for net file
    :return:
    """""
    pass


def generate_Visum_network_net(num_routes, max_stops_per_route, net_file):
    """
    Generate a Visum timetable and save it as .net file

    :param num_routes:
    :param max_stops_per_route:
    :param net_file:
    :return:
    """

    all_stops = get_all_stop_no()
    bus_routes = generate_bus_routes(all_stops, num_routes, max_stops_per_route)
    bus_nr = []
    for i in range(num_routes):
        if i >= 10:
            bus_nr.append("1" + str(i + 10))
        else:
            bus_nr.append("11" + str(i))
    save_as_net_file(bus_routes, bus_nr, net_file)


#### Run several times with different methods to test run time
##### Read all attributes:

In [6]:
all_stops = get_all_stop_no()
print(all_stops)  # Print the list of all line route names


[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0]


##### Generate lines in Visum:

In [87]:
# Original
remove_line_routes()
generate_Visum_network(16, 6)

In [7]:
# turn off redraw
Visum.Graphic.StopDrawing = True
remove_line_routes()
generate_Visum_network(16, 6)
Visum.Graphic.StopDrawing = False

#### Example of loading a net file
The actual creation of the net file is not implemented.
I have added comments inside the example net file. _Mandl_test_fewer_attributes.net_
Hopefully you can understand the meaning of all attributes.
The example is for a timetable based assignment.
If we use a frequency based assignment instead, we can skip all vehicle journeys but need to think of how we define headway and matched transfer

In [11]:
# Loading .NET file
remove_line_routes()

# Example of a net file. I have added some comments about attribute inside the file
net_file = os.path.join(work_path, 'Mandl_test_fewer_attributes.net')

# Generate timetable. Do not actually do anything yet. Just a stub
generate_Visum_network_net(16, 6, net_file)

# Loads the file to Visum
load_net_file(net_file)

In [90]:
# remove_line_routes()

In [91]:
#Run Visum assignment
Visum.Procedures.Execute()

In [92]:
demand = helpers.GetSkimMatrixRaw(Visum, 1)
df_demand = pd.DataFrame(demand)
jrt = helpers.GetSkimMatrixRaw(Visum, 2)
df_jrd = pd.DataFrame(jrt)

In [93]:
df_demand

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,400.0,200.0,60.0,80.0,150.0,75.0,75.0,30.0,160.0,30.0,25.0,35.0,0.0,0.0
1,400.0,0.0,50.0,120.0,20.0,180.0,90.0,90.0,15.0,130.0,20.0,10.0,10.0,5.0,0.0
2,200.0,50.0,0.0,40.0,60.0,180.0,90.0,90.0,15.0,45.0,20.0,10.0,10.0,5.0,0.0
3,60.0,120.0,40.0,0.0,50.0,100.0,50.0,50.0,15.0,240.0,40.0,25.0,10.0,5.0,0.0
4,80.0,20.0,60.0,50.0,0.0,50.0,25.0,25.0,10.0,120.0,20.0,15.0,5.0,0.0,0.0
5,150.0,180.0,180.0,100.0,50.0,0.0,100.0,100.0,30.0,880.0,60.0,15.0,15.0,10.0,0.0
6,75.0,90.0,90.0,50.0,25.0,100.0,0.0,50.0,15.0,440.0,35.0,10.0,10.0,5.0,0.0
7,75.0,90.0,90.0,50.0,25.0,100.0,50.0,0.0,15.0,440.0,35.0,10.0,10.0,5.0,0.0
8,30.0,15.0,15.0,15.0,10.0,30.0,15.0,15.0,0.0,140.0,20.0,5.0,0.0,0.0,0.0
9,160.0,130.0,45.0,240.0,120.0,880.0,440.0,440.0,140.0,0.0,600.0,250.0,500.0,200.0,0.0


In [94]:
df_jrd


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,999999.000000,999999.000000,999999.00,999999.0,999999.0,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000,999999.0,999999.000000,999999.000000,999999.000000
1,999999.0,0.000000,187.400000,999999.00,999999.0,999999.0,174.600000,999999.000000,454.966667,157.583333,305.216667,999999.0,216.283333,999999.000000,999999.000000
2,999999.0,187.400000,0.000000,999999.00,999999.0,999999.0,281.000000,999999.000000,561.366667,263.983333,411.616667,999999.0,322.683333,999999.000000,999999.000000
3,999999.0,999999.000000,999999.000000,0.00,999999.0,999999.0,999999.000000,234.700000,999999.000000,999999.000000,999999.000000,999999.0,999999.000000,177.600000,337.550000
4,999999.0,999999.000000,999999.000000,999999.00,0.0,999999.0,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000,999999.0,999999.000000,999999.000000,999999.000000
5,999999.0,999999.000000,999999.000000,999999.00,999999.0,0.0,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000,999999.0,999999.000000,999999.000000,999999.000000
6,999999.0,174.600000,281.000000,999999.00,999999.0,999999.0,0.000000,999999.000000,342.150000,93.950000,192.400000,999999.0,103.466667,999999.000000,999999.000000
7,999999.0,999999.000000,999999.000000,234.70,999999.0,999999.0,999999.000000,0.000000,999999.000000,999999.000000,999999.000000,999999.0,999999.000000,132.533333,207.683333
8,999999.0,458.183333,564.583333,999999.00,999999.0,999999.0,340.200000,999999.000000,0.000000,377.533333,393.966667,999999.0,276.800000,999999.000000,999999.000000
9,999999.0,157.583333,263.983333,999999.00,999999.0,999999.0,93.950000,999999.000000,374.316667,0.000000,224.566667,999999.0,135.633333,999999.000000,999999.000000


In [95]:
#these are copied from my other note book and co calculations does not work yet...

def calculate_cp_score(travel_time_matrix, demand_matrix):
    """
    Calculate the CP score for a public transport network.
    
    Parameters:
    - travel_time_matrix: DataFrame, travel times with penalty included between all O-D pairs.
    - demand_matrix: DataFrame, demand between all O-D pairs.
    
    Returns:
    - CP_score: float, the calculated CP score of the network.
    """
    travel_time_matrix = travel_time_matrix.replace(999999, 1000)
    # Multiply travel times by demands for each O-D pair
    weighted_travel_times = travel_time_matrix * demand_matrix

    # Sum the weighted travel times for the entire network
    total_weighted_travel_time = weighted_travel_times.sum().sum()

    # Sum the total demands for the network
    total_demand = demand_matrix.sum().sum()

    # Calculate the CP score
    CP_score = total_weighted_travel_time / total_demand

    return CP_score


import networkx as nx


def calculate_co_score(routes, travel_time_matrix, print_time=False):
    G = nx.Graph()
    total_time = 0
    visited_stations = set()
    penalty = 0
    # Build the graph from the routes
    for route in routes:
        route_time = 0
        for i in range(len(route) - 1):
            from_node = route[i]
            to_node = route[i + 1]
            visited_stations.update([from_node, to_node])
            G.add_edge(from_node, to_node)

            travel_time = travel_time_matrix.loc[from_node, to_node]
            route_time += travel_time

            if print_time:
                print(f"Travel time from {from_node} to {to_node}: {travel_time}")

        total_time += route_time

    # Check if the network is fully connected
    all_stations = set(travel_time_matrix.columns)

    # Remove visited stations from the set of all stations to find unvisited ones
    unvisited_stations = all_stations - visited_stations
    #print(unvisited_stations)
    penalty += len(unvisited_stations) * 100
    if not nx.is_connected(G):
        # If not, apply a penalty
        penalty += 100

        if print_time:
            print(f"Penalty for unconnected network: {penalty}")
    total_time += penalty
    return total_time


def calculate_score(cp, co):
    return cp

In [96]:
cp_score = calculate_cp_score(df_jrd, df_demand)
cp_score

750.3259419824449